In [ ]:
!pip install --upgrade langchain
!pip install python-dotenv
!pip install openai==0.28.0

In [ ]:
from dotenv import load_dotenv
import os
import openai
import json

load_dotenv()
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [ ]:
def get_pizza_info(pizza_name: str):
    piza_info = {
        "name": pizza_name,
        "price": "10.99"
    }
    return json.dumps(piza_info)

In [ ]:
functions = [
    {
        #name is the name of the function that I defined
        "name": "get_pizza_info",
        "description": "Get name and price of a pizza of the restaurant",
        "parameters": {
            "type": "object",
            "properties": {
                # the parameter name in the function
                "pizza_name": {
                    "type": "string",
                    "description": "The name of the pizza, e.g. Salami",
                },
            },
            # pizza_name is required since we dont have any default value for that in the function
            "required": ["pizza_name"],
        },
    }
]

In [ ]:
import openai
my_key = openai.api_key
def chat(query):
    # Set your API key here (ideally it should be set as an environment variable for security reasons)
    openai.api_key = my_key

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": query}],
        functions = functions
    )

    # Assuming the response structure is correct and you're looking for a message in the response
    message = response['choices'][0]['message']
    return message

# Example call
print(chat("What is the capital of Turkey?"))
chat("What is the capital of France?")


In [ ]:
query = "How much does pizza salami cost?"
message = chat(query)
message

In [ ]:
query = "How much does pizza salami cost?"
if message.get("function_call"):
    function_name = message["function_call"]["name"]

    pizza_name = json.loads(message["function_call"]["arguments"]).get("pizza_name")

    function_response = get_pizza_info(
        pizza_name= pizza_name
    )

    second_response = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo",
        messages = [
            {"role": "user", "content": query},
            message,
            {
                "role": "function",
                "name": function_name,
                "content": function_response, # function_response is used to create a human like anwer with the information from our api
            },
        ],
    )


print(second_response)

In [ ]:
!pip install langchain-community langchain-core

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, ChatMessage

In [ ]:
llm = ChatOpenAI(model = "gpt-3.5-turbo")
message = llm.predict_messages(
    [HumanMessage(content = "What is teh capital of France?")], functions=functions
)
message

In [ ]:
query = "How much does Pizza Salami cost in the restaurant?"
message_pizza = llm.predict_messages([HumanMessage(content=query)], functions=functions)
message_pizza

In [ ]:
message.additional_kwargs

In [ ]:
message_pizza.additional_kwargs

In [ ]:
import json

pizza_name = json.loads(message_pizza.additional_kwargs["function_call"]["arguments"]).get("pizza_name")
pizza_name

In [ ]:
pizza_api_response = get_pizza_info(pizza_name=pizza_name)
pizza_api_response

In [ ]:
second_response = llm.predict_messages(
    [
        HumanMessage(content=query),
        AIMessage(content=str(message_pizza.additional_kwargs)),
        ChatMessage(
            role = "function",
            additional_kwargs={
                "name": message_pizza.additional_kwargs["function_call"]["name"]
            },
            content=pizza_api_response
        ),
    ],
    functions = functions,
)
second_response

### Use Tools
you can convert Tools to functions, both the Tools provided by langchain and also your own tools

In [ ]:
from typing import Optional
from langchain.tools import BaseTool
from langchain.callbacks.manager import(
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun
)

class StupidJokeTool(BaseTool):
    name = "StupidJokeTool"
    description = "Tool to explain jokes about chickens"

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        return "It is funny, because AI..."
    
    async def _arun(
        self, query:str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        raise NotImplementedError("joke tool does not support async")

In [ ]:
from langchain.tools import format_tool_to_openai_function, MoveFileTool

tools = [StupidJokeTool(), MoveFileTool()]
functions = [format_tool_to_openai_function(t) for t in tools]
functions

In [ ]:
query = "Why does the chicken cross the road? To get to the other side"
output = llm.predict_messages([HumanMessage(content=query)], functions=functions)
output

In [ ]:
question = json.loads(output.additional_kwargs["function_call"]["arguments"]).get("query")

tool_response = tools[0].run(question)
tool_response

In [ ]:
second_response = llm.predict_messages(
    [
        HumanMessage(content=query),
        AIMessage(content=str(output.additional_kwargs)),
        ChatMessage(
            role = "function",
            additional_kwargs= {
                "name": output.additional_kwargs["function_call"]["name"]
            },
            content="""
            {tool_response}""",
        ),
    ],
    functions = functions,
)
second_response

In [ ]:
!pip install numexpr

In [ ]:
from langchain import LLMMathChain
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, model= "gpt-3.5-turbo")

llm_math_chain = LLMMathChain.from_llm(llm = llm, verbose=True)

tools = [
    Tool(
        name = "Calculator",
        func = llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
]

In [ ]:
agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

In [ ]:
agent.run("what is the capital of France?")

In [ ]:
agent.run("what is 100 divided by 25?")